In [1]:
model_name = "answerdotai/ModernBERT-base" # max token = 8192
!pip install transformers datasets accelerate -q
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch
from torch.utils.data import Dataset
import os
os.environ["WANDB_DISABLED"] = "true"

# ----------------------------
# Dataclass set and tokeniztaion
# ----------------------------
class PairDataset(Dataset):
  def __init__(self, df, tokenizer, field1, field2, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.field1 = field1
        self.field2 = field2
        self.max_length = max_length

  def __len__(self):
      return len(self.df)

  def __getitem__(self, idx):
      row = self.df.iloc[idx]
      encoded = self.tokenizer(
          row[self.field1],
          row[self.field2],
          truncation=True,
          padding="max_length",
          max_length=self.max_length,
          return_tensors="pt",
      )
      return {
          "input_ids": encoded["input_ids"].squeeze(),
          "attention_mask": encoded["attention_mask"].squeeze(),
          "labels": torch.tensor(row["label"], dtype=torch.long),
      }

# ----------------------------
# metirc help function
# ----------------------------
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  return {
      "accuracy": accuracy_score(labels, preds),
      "f1": f1_score(labels, preds),
  }

# ----------------------------
# train_and_save help function
# ----------------------------
def train_and_save_modernbert(train_df, val_df, field1, field2, save_name,
                              max_length=2048, epochs=1, batch=8):

  model_name = "answerdotai/ModernBERT-base" #ModernBERT

  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

  train_dataset = PairDataset(train_df, tokenizer, field1, field2, max_length)
  val_dataset   = PairDataset(val_df, tokenizer, field1, field2, max_length)

  model = AutoModelForSequenceClassification.from_pretrained(
      model_name,
      num_labels=2,
      trust_remote_code=True
  )

  training_args = TrainingArguments(
      output_dir="./modernbert-checkpoints",
      eval_strategy="epoch",
      save_strategy="epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=batch,
      per_device_eval_batch_size=batch,
      num_train_epochs=epochs,
      weight_decay=0.01,
      load_best_model_at_end=True,
      metric_for_best_model="f1",
      fp16=True,
      report_to="none",
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  # save BEST model
  save_path = f"/content/drive/MyDrive/266NoteBooks/FinalProject/Model/{save_name}"
  trainer.save_model(save_path)
  tokenizer.save_pretrained(save_path)

  print("Model saved to:", save_path)
  return save_path

# ----------------------------
# load_and_eval help function
# ----------------------------
def load_and_eval_modernbert(model_path, test_df, field1, field2, max_length=2048):

  tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
  model = AutoModelForSequenceClassification.from_pretrained(
      model_path, trust_remote_code=True
  )

  test_dataset = PairDataset(test_df, tokenizer, field1, field2, max_length)

  trainer = Trainer(
      model=model,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
  )

  raw = trainer.evaluate(test_dataset)

  metrics = {
      "test_loss": raw["eval_loss"],
      "test_accuracy": raw["eval_accuracy"],
      "test_f1": raw["eval_f1"],
  }

  print("\nTest Metrics:", metrics)
  return metrics


# ----------------------------
# Run pipeline (train-save-load-eval OR load-eval)
# ----------------------------
def run_modernbert_pipeline(
        pair_type, field1, field2, save_name,
        max_length=2048, epochs=1, batch=8,
        train_first=True):

  data_root = '/content/drive/MyDrive/266NoteBooks/FinalProject/Data/'
  model_root = "/content/drive/MyDrive/266NoteBooks/FinalProject/Model"
  model_path = f"{model_root}/{save_name}"

  train_df = pd.read_json(f"{data_root}{pair_type}/train.jsonl", lines=True)
  val_df   = pd.read_json(f"{data_root}{pair_type}/val.jsonl", lines=True)
  test_df  = pd.read_json(f"{data_root}{pair_type}/test.jsonl", lines=True)

  print(f"\nLoaded dataset: {pair_type}")
  print("Train:", train_df.shape, "Val:", val_df.shape, "Test:", test_df.shape)

  if train_first:
      print(f"\nTraining ModernBERT → {save_name}")
      model_path = train_and_save_modernbert(
          train_df=train_df,
          val_df=val_df,
          field1=field1,
          field2=field2,
          save_name=save_name,
          max_length=max_length,
          epochs=epochs,
          batch=batch
      )
  else:
      print(f"\nSkipping training. Load only: {model_path}")

  print(f"\nEvaluating ModernBERT: {save_name}")
  metrics = load_and_eval_modernbert(
      model_path=model_path,
      test_df=test_df,
      field1=field1,
      field2=field2,
      max_length=max_length
  )

  print("\nFinished:", save_name)
  return metrics

In [2]:
# Title
run_modernbert_pipeline("title-title-pair","title1","title2","modernbert-title-v1",
                        max_length=64, epochs=1, batch=64, train_first=True)


Loaded dataset: title-title-pair
Train: (489640, 3) Val: (58976, 3) Test: (59660, 3)

Training ModernBERT → modernbert-title-v1


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.058400,0.072117,0.974735,0.974643


Model saved to: /content/drive/MyDrive/266NoteBooks/FinalProject/Model/modernbert-title-v1

Evaluating ModernBERT: modernbert-title-v1


/tmp/ipython-input-960418307.py:119: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Test Metrics: {'test_loss': 0.06274166703224182, 'test_accuracy': 0.9764498826684546, 'test_f1': 0.9763766288356452}

Finished: modernbert-title-v1


{'test_loss': 0.06274166703224182,
 'test_accuracy': 0.9764498826684546,
 'test_f1': 0.9763766288356452}

In [4]:
# Body
run_modernbert_pipeline("body-body-pair","body1","body2","modernbert-body1024-v1",
                        max_length=1024, epochs=1, batch=8, train_first=True)


Loaded dataset: body-body-pair
Train: (402588, 3) Val: (48390, 3) Test: (49514, 3)

Training ModernBERT → modernbert-body1024-v1


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-960418307.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.036300,0.054225,0.986505,0.986475


Model saved to: /content/drive/MyDrive/266NoteBooks/FinalProject/Model/modernbert-body1024-v1

Evaluating ModernBERT: modernbert-body1024-v1


/tmp/ipython-input-960418307.py:119: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Test Metrics: {'test_loss': 0.050778020173311234, 'test_accuracy': 0.9872965221957426, 'test_f1': 0.9872875361264375}

Finished: modernbert-body1024-v1


{'test_loss': 0.050778020173311234,
 'test_accuracy': 0.9872965221957426,
 'test_f1': 0.9872875361264375}

In [5]:
# Post
run_modernbert_pipeline("post-post-pair","post1","post2","modernbert-post1024-v1",
                        max_length=1024, epochs=1, batch=8, train_first=True)



Loaded dataset: post-post-pair
Train: (402276, 3) Val: (46504, 3) Test: (52104, 3)

Training ModernBERT → modernbert-post1024-v1


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-960418307.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.026700,0.032060,0.992753,0.992748


Model saved to: /content/drive/MyDrive/266NoteBooks/FinalProject/Model/modernbert-post1024-v1

Evaluating ModernBERT: modernbert-post1024-v1


/tmp/ipython-input-960418307.py:119: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Test Metrics: {'test_loss': 0.028509771451354027, 'test_accuracy': 0.9932442806694304, 'test_f1': 0.9932401290520817}

Finished: modernbert-post1024-v1


{'test_loss': 0.028509771451354027,
 'test_accuracy': 0.9932442806694304,
 'test_f1': 0.9932401290520817}